In [2]:
%pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.3 MB 18.8 MB/s eta 0:00:01
   -------------------------- ------------- 8.1/12.3 MB 28.5 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 29.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import duckdb